In [1]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [2]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if DCBS[lm[0]].__contains__(code):
                if low_prc(code, dt, lm, fcd[1]):
                    satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DCBS[dt][code]['dp']):
                lm = calculate_time(dt, 'p', 20)
                if DCBS[lm[0]].__contains__(code):
                    prc = get_prcs(code, lm)
                    lp = absolute_low(prc, 0)
                    rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]

def get_prcs(code, dts):
    prc = []
    
    for dt in dts:
        prc.append(DCBS[dt][code]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DCBS[dt][code]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = list(DCBS.keys())
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]



In [3]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
td
DCBS.keys()


dict_keys(['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08'])

In [4]:
select = list(DCBS[td].keys())

flt_cd = [['dp', 100, 130]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]









flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]


flt_cd = [['dp', 100, 130], ['bl',0, 3]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]

flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
flt_cd = [['dp', 100, 130], ['bl',0, 3]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]
for fcd in flt_cd:
    select = select_condition(select, td, fcd)

print(len(select))
    
# rk_cd = [['lp', 10, 'a']]
rk_cd = [['dl', 600, 'a']]



rk_cd = [['dl', 30, 'a']]



rk_cd = [['dl', 30, 'a']]






rk_cd = [['lp', 30, 'a']]


rk_cd = [['ytm', 20, 'd']]

rk_cd = [['dl', 30, 'a']]
codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(codes)

24


24

In [5]:
for r in codes:
    print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "收盘价：", DCBS[td][r]['dp'],  "转股溢价率：", DCBS[td][r]['cpr'], "到期收益率：", DCBS[td][r]['ytm'],"换手率：", DCBS[td][r]['trt'], '市值',  DCBS[td][r]['bl'], '双低', DCBS[td][r]['dl'])

118016.SH 京源转债 688096.SH 京源环保 收盘价： 116.543 转股溢价率： 19.29 到期收益率： 0.86 换手率： 2.18 市值 3.33 双低 135.833
113628.SH 晨丰转债 603685.SH 晨丰科技 收盘价： 125.993 转股溢价率： 12.91 到期收益率： -0.94 换手率： 4.91 市值 4.15 双低 138.903
128066.SZ 亚泰转债 002811.SZ 郑中设计 收盘价： 115.898 转股溢价率： 25.34 到期收益率： -0.89 换手率： 3.25 市值 4.61 双低 141.238
123160.SZ 泰福转债 300992.SZ 泰福泵业 收盘价： 118.186 转股溢价率： 23.52 到期收益率： 0.53 换手率： 2.73 市值 3.349 双低 141.70600000000002
123063.SZ 大禹转债 300021.SZ 大禹节水 收盘价： 126.583 转股溢价率： 15.8 到期收益率： -0.28 换手率： 2.55 市值 6.346 双低 142.383
123162.SZ 东杰转债 300486.SZ 东杰智能 收盘价： 125.965 转股溢价率： 17.1 到期收益率： -0.66 换手率： 3.49 市值 5.633 双低 143.065
127060.SZ 湘佳转债 002982.SZ 湘佳股份 收盘价： 111.071 转股溢价率： 32.99 到期收益率： 0.54 换手率： 5.49 市值 6.4 双低 144.061
123167.SZ 商络转债 300975.SZ 商络电子 收盘价： 126.587 转股溢价率： 22.16 到期收益率： -0.77 换手率： 5.01 市值 3.965 双低 148.747
110070.SH 凌钢转债 600231.SH 凌钢股份 收盘价： 115.671 转股溢价率： 37.43 到期收益率： -0.02 换手率： 2.46 市值 2.17 双低 153.101
113665.SH 汇通转债 603176.SH 汇通集团 收盘价： 112.475 转股溢价率： 44.4 到期收益率： 1.25 换手率： 2.45 市值 3.6 双低 156.875
113565.SH 宏辉转债

In [41]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
select = list(DCBS[td].keys())
for code in select:
    DCBS[td][code]['cat1'] = DTBS['B'][code]['cat1']
    DCBS[td][code]['cat2'] = DTBS['B'][code]['cat2']
    
# save
f_save = open(DCBS_path, 'wb')
pickle.dump(DCBS, f_save)
f_save.close()


In [34]:
DTBS['127082.SZ']

KeyError: '127082.SZ'

In [9]:
cbcodes = []
stcodes = []

cbcodes = list(DTBS['A'].keys())

for cb in cbcodes:
    st = DTBS['B'][cb]['sc']
    if st not in stcodes:
        stcodes.append(st)

In [11]:
len(stcodes)

722

In [13]:
df = pd.DataFrame(cbcodes)

In [15]:
df2 = pd.DataFrame(stcodes)

In [16]:
df.to_csv()

',0\r\n0,110088.SH\r\n1,113545.SH\r\n2,113631.SH\r\n3,123128.SZ\r\n4,123156.SZ\r\n5,127027.SZ\r\n6,127061.SZ\r\n7,110047.SH\r\n8,110063.SH\r\n9,110070.SH\r\n10,110087.SH\r\n11,110089.SH\r\n12,110092.SH\r\n13,111001.SH\r\n14,111002.SH\r\n15,111003.SH\r\n16,111007.SH\r\n17,111012.SH\r\n18,113025.SH\r\n19,113027.SH\r\n20,113046.SH\r\n21,113047.SH\r\n22,113058.SH\r\n23,113059.SH\r\n24,113064.SH\r\n25,113534.SH\r\n26,113535.SH\r\n27,113566.SH\r\n28,113591.SH\r\n29,113600.SH\r\n30,113615.SH\r\n31,113621.SH\r\n32,113623.SH\r\n33,113632.SH\r\n34,113636.SH\r\n35,113638.SH\r\n36,113640.SH\r\n37,113641.SH\r\n38,113649.SH\r\n39,113650.SH\r\n40,113653.SH\r\n41,113657.SH\r\n42,113663.SH\r\n43,118000.SH\r\n44,118005.SH\r\n45,118006.SH\r\n46,118008.SH\r\n47,118010.SH\r\n48,118018.SH\r\n49,118020.SH\r\n50,118022.SH\r\n51,118023.SH\r\n52,118028.SH\r\n53,118029.SH\r\n54,123012.SZ\r\n55,123013.SZ\r\n56,123031.SZ\r\n57,123046.SZ\r\n58,123052.SZ\r\n59,123057.SZ\r\n60,123076.SZ\r\n61,123078.SZ\r\n62,123085.S

In [19]:
df.to_csv('cb.csv')
df2.to_csv('st.csv')